# D7047E Advanced Deep Learning
## Lab 0
Refresher on PyTorch for course exercises and project

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter 

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # NVIDIA GPU
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")   # Apple GPU (MPS)
else:
    device = torch.device("cpu")   # Fallback to CPU
print(f"Using device: {device}")

Using device: mps


### 0.1 — CNN uisng CIFAR-10

In [10]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform

)

train_loader = torch.utils.data.DataLoader(
    train_set, 
    batch_size=32, 
    shuffle=True,
    num_workers=2
)

test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

test_loader = torch.utils.data.DataLoader(
    test_set, 
    batch_size=32, 
    shuffle=False,
    num_workers=2
)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [12]:
print(f"Training set size: {len(train_set)}")
print(f"Test set size: {len(test_set)}")

# Example: Iterate through one batch to check
dataiter = iter(train_loader)
images, labels = next(dataiter)
print(f"Batch shape: {images.shape}")  # Should be [batch_size, 3, 32, 32]
print(f"Labels: {labels}")

Training set size: 50000
Test set size: 10000
Batch shape: torch.Size([32, 3, 32, 32])
Labels: tensor([3, 7, 6, 1, 4, 6, 2, 7, 5, 9, 4, 6, 7, 9, 0, 3, 0, 4, 5, 2, 1, 9, 5, 7,
        4, 2, 1, 9, 4, 3, 6, 3])


In [18]:
class CNN(nn.Module):
    def __init__(self, activation):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.activation = activation
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)  # No activation on output
        return x


In [ ]:
def train_and_evaluate(model: nn.Module, train_loader: DataLoader, test_loader: DataLoader,
                       optimizer: optim.Optimizer, criterion: nn.Module, device: torch.device,
                       writer: SummaryWriter, experiment_name: str, num_epochs: int = 10) -> float:
    print(f"Starting {experiment_name}")
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(train_loader):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()
            global_step = epoch * len(train_loader) + i
            writer.add_scalar(f"Loss/Train/{experiment_name}", loss.item(), global_step)

            if i % 200 == 199:
                avg_loss = running_loss / 200
                print(f"[{experiment_name}] Epoch {epoch + 1}, Batch {i + 1}, Loss: {avg_loss:.3f}")
                running_loss = 0.0

        train_accuracy = 100 * correct / total
        writer.add_scalar(f"Accuracy/Train/{experiment_name}", train_accuracy, epoch)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    writer.add_scalar(f"Accuracy/Test/{experiment_name}", test_accuracy, num_epochs)
    print(f"{experiment_name} - Test Accuracy: {test_accuracy:.2f}%")
    return test_accuracy

In [ ]:
def main():

    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    print(f"Using device: {device}")

    activations = {
        "LeakyReLU": nn.LeakyReLU(negative_slope=0.1),
        "Tanh": nn.Tanh()
    }
    optimizers = {
        "SGD": lambda params: optim.SGD(params, lr=0.0001),
        "Adam": lambda params: optim.Adam(params, lr=0.0001)
    }

    criterion = nn.CrossEntropyLoss()
    for act_name, activation in activations.items():
        for opt_name, opt_fn in optimizers.items():
            experiment_name = f"{act_name}_{opt_name}"
            writer = SummaryWriter(log_dir=f"runs/{experiment_name}")
            model = CNN(activation=activation).to(device)
            optimizer = opt_fn(model.parameters())
            train_and_evaluate(model, train_loader, test_loader, optimizer, criterion, device, writer, experiment_name)
            writer.close()

    print("All experiments completed")

if __name__ == "__main__":
    main()

Using device: mps
Starting LeakyReLU_SGD
[LeakyReLU_SGD] Epoch 1, Batch 200, Loss: 2.303
[LeakyReLU_SGD] Epoch 1, Batch 400, Loss: 2.305
[LeakyReLU_SGD] Epoch 1, Batch 600, Loss: 2.304
[LeakyReLU_SGD] Epoch 1, Batch 800, Loss: 2.303
[LeakyReLU_SGD] Epoch 1, Batch 1000, Loss: 2.302
[LeakyReLU_SGD] Epoch 1, Batch 1200, Loss: 2.301
[LeakyReLU_SGD] Epoch 1, Batch 1400, Loss: 2.303
[LeakyReLU_SGD] Epoch 2, Batch 200, Loss: 2.302
[LeakyReLU_SGD] Epoch 2, Batch 400, Loss: 2.301
[LeakyReLU_SGD] Epoch 2, Batch 600, Loss: 2.302
[LeakyReLU_SGD] Epoch 2, Batch 800, Loss: 2.302
[LeakyReLU_SGD] Epoch 2, Batch 1000, Loss: 2.301
[LeakyReLU_SGD] Epoch 2, Batch 1200, Loss: 2.300
[LeakyReLU_SGD] Epoch 2, Batch 1400, Loss: 2.299
[LeakyReLU_SGD] Epoch 3, Batch 200, Loss: 2.300
[LeakyReLU_SGD] Epoch 3, Batch 400, Loss: 2.300
[LeakyReLU_SGD] Epoch 3, Batch 600, Loss: 2.300
[LeakyReLU_SGD] Epoch 3, Batch 800, Loss: 2.299
[LeakyReLU_SGD] Epoch 3, Batch 1000, Loss: 2.298
[LeakyReLU_SGD] Epoch 3, Batch 1200, Los

### 0.2 — Transfer Learning

#### 0.2.1 — Transfer Learning from ImageNet
Using AlexNet


#### 0.2.2 — Transfer Learning from MNIST